# <font color='blue'> Step 1 - Scraping </font>

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Nasa Mars News

In [4]:
#URL for NASA Mars News
url = 'https://redplanetscience.com/'
browser.visit(url)

In [5]:
# HTML object
html = browser.html
    
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Retrieve all elements that has article information
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA to Reveal Name of Its Next Mars Rover
After a months-long contest among students to name NASA's newest Mars rover, the agency will reveal the winning name — and the winning student — this Thursday. 


## JPL Mars Space Images - Featured Image

In [6]:
#URL for JPL Mars Space Images - Featured Image
space_url = 'https://spaceimages-mars.com/'
browser.visit(space_url)

In [7]:
#Use Splinter to navigate the site to find Featured Mars Image
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Retrieve all elements that article information
div = soup.find('div', class_='header')
a = div.find('a', class_='showimg fancybox-thumbs')
href = a['href']
featured_image_url = str(space_url) + str(href)
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [8]:
browser.quit()

## Mars Facts

In [9]:
#Use pandas to convert the data to a HTML table string
mars_facts_url = 'https://galaxyfacts-mars.com'
tables = pd.read_html(mars_facts_url)
mars_facts_df = tables[0]
mars_facts_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
mars_facts_df.columns = mars_facts_df.iloc[0]
cleaned_df = mars_facts_df.drop(mars_facts_df.index[0])
cleaned_df.set_index('Mars - Earth Comparison', inplace=True)
cleaned_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [11]:
#Convert data to HTML table string
html_table = cleaned_df.to_html()
html_table.replace('\n', '')
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [12]:
cleaned_df.to_html('table.html')

## Mars Hemispheres

In [13]:
#Mars Hemispheres
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

hem_url = 'https://marshemispheres.com/'
browser.visit(hem_url)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\silve\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [14]:
#Mars Hemispheres
html = browser.html
    
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

hemisphere_image_urls = []

# Retrieve all elements that article information
section = soup.find('section', id='results-accordian')

#Retrieve the individual "product" info
div = section('div', class_='description')

for product in div:
    #Create dictionary to store title & img url
    dict = {}

    #Gets the title by finding the h3 tag
    title = product.find('h3').text
    dict['title'] = title
    #print(title)

    #Clicks the link with that title
    browser.links.find_by_partial_text(title).click()

    #Sets the html to the new page's html
    html = browser.html
    soup = bs(html, 'html.parser')

    #Clicks the "Open" to get the full image
    browser.links.find_by_partial_text('Open').click()
    img = soup.find('img', class_='wide-image')['src']
    #Adds the the first part of the img url
    img_url = str(hem_url + img)
    dict['img_url'] = img_url

    hemisphere_image_urls.append(dict)

    #Returns & resets to main URL to get next info
    browser.visit(hem_url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
browser.quit()

In [15]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [17]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title" : news_title,
    "news_p" : news_p,
    "featured_image_url" : featured_image_url,
    "mars_facts" : html_table,
    "hemisphere_images" : hemisphere_image_urls
}

In [18]:
mars_dict

{'news_title': 'NASA to Reveal Name of Its Next Mars Rover',
 'news_p': "After a months-long contest among students to name NASA's newest Mars rover, the agency will reveal the winning name — and the winning student — this Thursday. ",
 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n 

In [19]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [20]:
db = client.marsdata_db
marsdata = db.marsdata
marsdata.insert_one(mars_dict)